# 本題要找出每個使用者被提及的次數 (\#Array, \#Heap, \#HashMap )
原題目連結: https://leetcode.com/problems/count-mentions-per-user/description/?envType=daily-question&envId=2025-12-12  

You are given an integer numberOfUsers representing the total number of users and an array events of size n x 3.  
Each events[i] can be either of the following two types:  
- Message Event: ["MESSAGE", "timestampi", "mentions_stringi"]  
    - This event indicates that a set of users was mentioned in a message at timestampi.  
    - The mentions_stringi string can contain one of the following tokens:   
          id<number>: where <number> is an integer in range [0,numberOfUsers - 1]. There can be multiple ids separated by a single whitespace and may contain duplicates. This can mention even the offline users.
  
    - ALL: mentions all users.  
    - HERE: mentions all online users.
- Offline Event: ["OFFLINE", "timestampi", "idi"]  
This event indicates that the user idi had become offline at timestampi for 60 time units. The user will automatically be online again at time timestampi + 60.
Return an array mentions where mentions[i] represents the number of mentions the user with id i has across all MESSAGE events.

All users are initially online, and if a user goes offline or comes back online, their status change is processed before handling any message event that occurs at the same timestamp.  
Note that a user can be mentioned multiple times in a single message event, and each mention should be counted separately.  

範例:  
Ex1:  
Input: numberOfUsers = 2, events = [["MESSAGE","10","id1 id0"],["OFFLINE","11","0"],["MESSAGE","71","HERE"]]  
Output: [2,2]  
Explanation:  
Initially, all users are online.  
At timestamp 10, id1 and id0 are mentioned. mentions = [1,1]  
At timestamp 11, id0 goes offline.    
At timestamp 71, id0 comes back online and "HERE" is mentioned. mentions = [2,2]  

Ex2:  
Input: numberOfUsers = 2, events = [["MESSAGE","10","id1 id0"],["OFFLINE","11","0"],["MESSAGE","12","ALL"]]  
Output: [2,2]  
Explanation:  
Initially, all users are online.  
At timestamp 10, id1 and id0 are mentioned. mentions = [1,1]  
At timestamp 11, id0 goes offline.  
At timestamp 12, "ALL" is mentioned. This includes offline users, so both id0 and id1 are mentioned. mentions = [2,2]  

Ex3:  
Input: numberOfUsers = 2, events = [["OFFLINE","10","0"],["MESSAGE","12","HERE"]]  
Output: [0,1]  
Explanation:  
Initially, all users are online.  
At timestamp 10, id0 goes offline.  
At timestamp 12, "HERE" is mentioned. Because id0 is still offline, they will not be mentioned. mentions = [0,1]

* 解題想法:  
由於event有可能不是按照時間進行排序，因此需要先進行加工，這邊用heap來放處理過的event，並把message type轉成integer方便進行排序，因為題目有限制是狀態的改變優先於訊息，然後再從heap中開始取值，如果是offline的情況則放入dict中進行紀錄，如果是訊息則檢查訊息的內容有哪些:
    - ALL: 全部的id都加1
    - HERE: 需要檢查offline狀態，如果時間已經超過offline限制則刪除offline狀態並將該id次數加1
    - 其他則是用id去個別加1，這邊不受到offline限制
  
    最後回傳統計的結果就是答案

In [ ]:
import heapq

class Solution:
    def countMentions(self, numberOfUsers: int, events: List[List[str]]) -> List[int]:
        status = {}
        temp = []
        heapq.heapify(temp)
        for eve in events:
            if eve[0] == 'OFFLINE':
                heapq.heappush(temp, [int(eve[1]),0 , eve[2]])
            else:
                heapq.heappush(temp, [int(eve[1]),1 , eve[2]])
        count = [0] * numberOfUsers
        while temp:
            eve = heapq.heappop(temp)
            if eve[1] == 0:
                i = int(eve[-1])
                status[i] = int(eve[0]) + 60
            else:
                msg = eve[-1].split(' ')
                t = eve[0]
                for m in msg:
                    if m == 'ALL':
                        for i in range(numberOfUsers):
                            count[i] += 1
                    elif m == 'HERE':
                        for i in range(numberOfUsers):
                            if i in status.keys():
                                if status[i] <= t:
                                    status.pop(i)
                                    count[i] += 1
                            else:
                                count[i] += 1
                    else:
                        id = int(m[2:])
                        count[id] += 1
        return count